In [2]:
import re 
import os
import json
import pandas
import shutil 
import psycopg2
from bs4 import BeautifulSoup

from pyspark.sql import Row
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType
from pyspark.sql.functions import udf,col,regexp_extract,regexp_replace


sc = SparkSession.builder.getOrCreate()
sqlsc = SQLContext(sc)

In [2]:
def clean_timestamp(timestamp):
    months = {
        "janvier":"01",
        "février":"02",
        "mars":"03",
        "avril":"04",
        "mai":"05",
        "juin":"06",
        "juillet":"07",
        "août":"08",
        "septembre":"09",
        "octobre":"10",
        "novembre":"11",
        "décembre":"12"}
    day_regex  = "(\d{2})\s[^à]" #first group only
    year_regex = "\d{4}"
    time_regex = "\d{2}:\d{2}:\d{2}"
    day_regex  = re.compile(day_regex)
    year_regex = re.compile(year_regex)
    time_regex = re.compile(time_regex)
    day = day_regex.search(timestamp).group(1)
    year = year_regex.search(timestamp).group(0)
    time = time_regex.search(timestamp).group(0).replace(":","")
    final_timestamp = f"{day}-{month}-{year}-{time}"
    return final_timestamp
    
def clean_post_text(post_text):
    #Parsing html
    clean_text = ""
    soup = BeautifulSoup(post_text,"html.parser")
    for tag in soup("blockquote.blockquote-jv"):
        tag.clear()
        
    #Boucler sur la liste d'élements à l'envers est le meilleur moment 
    for elem in list(soup)[::-1]:
        if elem.name == "p":
            clean_text += elem.get_text()
        else:
            break
    
    return clean_text

In [3]:
in_path  = "./../boucled_scrapers/spiders/out/posts"
out_path = "./../boucled_scrapers/spiders/out/posts/processed" 
posts_df = sqlsc.read.json(out_path)
timestamp_regex = "(\d{2})\s(\D{3,9})\s(\d{4})\s[à]\s(\d{2}:\d{2}:\d{2})"
#g1:day|g2:month|g3:year|g4:time
months = {
        "janvier":"01",
        "février":"02",
        "mars":"03",
        "avril":"04",
        "mai":"05",
        "juin":"06",
        "juillet":"07",
        "août":"08",
        "septembre":"09",
        "octobre":"10",
        "novembre":"11",
        "décembre":"12"}
months_udf = udf(lambda x : months[x],StringType())
posts_df = posts_df.withColumn("day", regexp_extract(col("timestamp"),timestamp_regex,1))
posts_df = posts_df.withColumn("month", regexp_extract(col("timestamp"),timestamp_regex,2))
posts_df = posts_df.withColumn("month",months_udf(col("month")))
posts_df = posts_df.withColumn("year", regexp_extract(col("timestamp"),timestamp_regex,3))
posts_df = posts_df.withColumn("time", regexp_extract(col("timestamp"),timestamp_regex,4))

udf_clean_text = udf(lambda x : clean_post_text(x),StringType())
posts_df = posts_df.withColumn("post_text",udf_clean_text(col("post_text")))
posts_df.show(5)

+--------------------+--------------+----+----------+--------------------+--------------------+--------------------+--------+---+-----+----+--------+
|                 _id|        author|page|   post_id|           post_text|           text_hash|           timestamp|topic_id|day|month|year|    time|
+--------------------+--------------+----+----------+--------------------+--------------------+--------------------+--------+---+-----+----+--------+
|6273aa04a5de58047...|CimerYokoTaro5|   6|1181250640|Édit: d’autant qu...|PHA+TGUgICAgICAgI...|05 mai 2022 à 12:...|69646416| 05|   05|2022|12:12:27|
|6273aa04a5de58047...| Kyle_READYDNT|   6|1181250720|\nL'OP si il a le...|PHA+TGUgICAgICAgI...|05 mai 2022 à 12:...|69646416| 05|   05|2022|12:13:28|
|6273aa04a5de58047...|  Covidboomer7|   6|1181250760|C'est toujours la...|PHA+TGUgICAgICAgI...|05 mai 2022 à 12:...|69646416| 05|   05|2022|12:13:47|
|6273aa04a5de58047...|CimerYokoTaro5|   6|1181250824|C’est un argument...|PHA+TGUgICAgICAgI...|05 ma